# Predicting credit default with SVM

In this notebook we show how one can predict whether a customer will default on the repayment of their credit card loans using support vector machines. We will use the prepared dataset from the EDA notebook for this.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from plotnine import *
import os
import datetime
import pickle

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [3]:
# first we read in the feather file with the input features
credit_card_df = pd.read_feather(os.path.join('..', 'dataset', 'modelling_dataset.feather'))

In [4]:
credit_card_df.head()

,id,limit_bal,sex,education,marriage,age,pay_1,pay_2,pay_3,pay_4,...,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default,max_delay,freq_delay
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,689,0,0,0,0,1,2,2
1,2,120000,2,2,2,26,-1,2,0,0,...,3261,0,1000,1000,1000,0,2000,1,2,5
2,3,90000,2,2,2,34,0,0,0,0,...,15549,1518,1500,1000,1000,1000,5000,0,0,6
3,4,50000,2,2,1,37,0,0,0,0,...,29547,2000,2019,1200,1100,1069,1000,0,0,6
4,5,50000,1,2,1,57,-1,0,-1,0,...,19131,2000,36681,10000,9000,689,679,0,0,4


# Creating the train, test, and validation splits

- **We split the dataset into train, validation, and test**
- We will set test aside, which will be used to evaluate the final performance of the model. Then we will use train and validation to fit the actual model.
- Note that since our dataset is imbalanced, we must make sure that the distribution of the target across train, validation, and test is balanced, otherwise we run the risk of e.g. not have sufficient default cases in the test set, for instance.
- The reason why we split the data into train, validation, and test is that we want to train the model and then evaluate the model accuracy on data **that the model has never seen**. This is the only way to accurately evaluate the predictive performance of a model.
- There are no hard-and-fast rules on how you have to split the data into train, validation, and test, only rules of thumb. The general idea is that you should have enough training examples to minimize the variance (uncertainty) on the model parameters, and enough test examples to minimize the variance (uncertainty) on the model performance metrics
- If there is enough data, the following algorithm to split into train/validation/test is a good one:
    - split the dataset into fitting data (80%) and test set (20%)
    - split the fitting data further into train set (80% of the fitting data) and validation set (20% of the fitting data)
- If there is not enough data, then crossvalidation is the only option in order to find the best model parameters

In [5]:
class TrainTestSplitter(object):
    '''Class to perform the split of the data into train, test, and validation.
    '''
    def __init__(self, train_frac=0.8, validation_frac=0.2, seed=1234):
        self.train_frac = train_frac
        self.validation_frac = validation_frac
        self.seed = seed
    
    def calculate_statistics(self):
        statistics = {}
        for i in ['train_set', 'test_set', 'validation_set']:
            split_stats = {}
            default_count = (getattr(self, i).groupby('default').size().reset_index())
            split_stats['N_defaults'] = (default_count.loc[lambda x: x.default ==1, 0].iloc[0])
            split_stats['percentage_total_defaults'] = split_stats['N_defaults']/self.total_n_defaults * 100
            split_stats['N_not_defaults'] = default_count.loc[lambda x: x.default == 0, 0].iloc[0]
            split_stats['percentage_total_not_defaults'] = split_stats['N_not_defaults']/self.total_n_not_defaults * 100
            statistics[i] = split_stats
        self.split_statistics = statistics

    def split_train_test(self, df):
        print("Generating the train/validation/test splits...")
        self.total_n_defaults = df.loc[lambda x: x.default == 1].shape[0]
        self.total_n_not_defaults = df.loc[lambda x: x.default == 0].shape[0]
        self.train_set = df.sample(frac=self.train_frac, random_state=self.seed)
        self.test_set = df.loc[lambda x: ~x.id.isin(self.train_set.id)].reset_index(drop=True)
        self.validation_set = self.train_set.sample(frac=self.validation_frac).reset_index(drop=True)
        self.train_set = self.train_set.loc[lambda x: ~x.id.isin(self.validation_set.id)].reset_index(drop=True)
        print("calculating the statistics...")
        self.calculate_statistics()
        print("split completed")

In [6]:
# create a fitting_splits object that will hold the train, validation, and test data
fitting_splits = TrainTestSplitter()
fitting_splits.split_train_test(credit_card_df)

Generating the train/validation/test splits...
calculating the statistics...
split completed


In [11]:
fitting_splits.test_set.shape

(6000, 27)

In [12]:
fitting_splits.split_statistics

{'train_set': {'N_defaults': 4281,
  'percentage_total_defaults': 64.5117540687161,
  'N_not_defaults': 14919,
  'percentage_total_not_defaults': 63.85464817668207},
 'test_set': {'N_defaults': 1307,
  'percentage_total_defaults': 19.695599758890896,
  'N_not_defaults': 4693,
  'percentage_total_not_defaults': 20.086457798322204},
 'validation_set': {'N_defaults': 1048,
  'percentage_total_defaults': 15.792646172393008,
  'N_not_defaults': 3752,
  'percentage_total_not_defaults': 16.05889402499572}}

# Dummification and scaling

Before fitting a model to our data, we need to deal with two data transformations that are **essential** to be able to obtain a performant model. These are **dummification** or **one-hot-encoding** and **scaling**.

### Dummification
- **Dummification** is used to transform categorical variables into numerical variables, since most machine learning models can only deal with numerical variables. You probably already encountered this in your statistics course since linear regression requires dummification.
- In dummification, we transform a categorical variable with N values into C_0, ..., C_N columns that take only 0 or 1 values. A row then has exactly one of these columns with 1, and the others with 0, namely, the column corresponding to the original value of the categorical variable for that row.
- Note that there are alternative ways to deal with categorical variables. One is to use ordinal encoding, which simply encodes the categorical variable as integers. In the above dataset, the categorical variables of education, gender, marriage, etc. are ordinally encoded
- Ordinal encoding, however, introduces an order between the categories which doesn't necessarily make sense and can cause problems for ML algorithms. For instance, if we have two categories "male" and "female", and we encode them as 0 and 1, then we are also encoding that "male" < "female" since 0 < 1. Which does not make sense and can lead to issues with the models, for instance predictions in-between classes
- In our dataset, one might argue that while there is a natural order for education, no such order exists for sex and marriage. Therefore, we are going to one-hot encode the sex and marriage variables

We will use the sklearn one hot encoder to encode the sex and marriage variables. The education variable we will leave as an ordinal (integer) variable. See
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html for documentation.

### Scaling
- **feature scaling** refers to the application of a transformation to the numerical variables in the dataset which normalizes the range of the numerical variables, ensuring that these ranges are the same
- this is performed because often numerical variables can have wildly different ranges; for instance, the "age" variable in the above dataset has a very different range from the "limit_bal" variable.
- this can cause problems to machine learning models, in particular those, like SVM, that rely on measuring distances in e.g. Euclidean spaces. Without scaling, features whose range is in the high numers (e.g. limit_bal) will come to dominate the distance measures used in the algorithms, and features with ranges in the low numbers (e.g. age) will have little to no impact on the model
- there are many ways to standardize variables (https://en.wikipedia.org/wiki/Feature_scaling). The most common are:
    - min/max scaling
    - mean normalization
    - standardization (z-score calculation)

sklearn provides various classes to perform feature scaling. We are going to use the StandardScaler, which performs z-score standardization. See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html for documentation.

In [361]:
# 1. fit a one hot encoder transformer to the TRAIN SET. The object we obtain will be reused later, to transform our data
# note: it is important that these transformers are fitted only on the train data.
one_hot_encoder = OneHotEncoder() # one hot encoder is a class instance
one_hot_encoder.fit(fitting_splits.train_set[['sex', 'marriage']]) # we can fit the encoder instance on the columns that need to be transformed.

OneHotEncoder()

In [362]:
# let's check what the encoded did
one_hot_encoder.categories_ # after we fit the encoder, the instance learns which values are present in each data column.
encoded_names = one_hot_encoder.get_feature_names_out() # with this method we can retrieve the names of the new dummy columns that have been computed
encoded_names
encoded_categories = one_hot_encoder.transform(fitting_splits.train_set[['sex', 'marriage']]).toarray() # at this point, we can use the fitted encoder to transform any array with a sex and marriage column; not just the training set, but also the validation or test set. The encoder will take that array of shape (M,2) and transform it to an array of shape (M, N), where N is the total number of distinct possible values for the encoded features. The values of the new array will be either 0 or 1, encoding whether that value of the feature applies to the row.
df_encoded = pd.DataFrame(encoded_categories)
df_encoded.columns = encoded_names
df_encoded

[array(['1', '2'], dtype=object), array(['0', '1', '2', '3'], dtype=object)]

array(['sex_1', 'sex_2', 'marriage_0', 'marriage_1', 'marriage_2',
       'marriage_3'], dtype=object)

,sex_1,sex_2,marriage_0,marriage_1,marriage_2,marriage_3
0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
19195,0.0,1.0,0.0,0.0,1.0,0.0
19196,0.0,1.0,0.0,0.0,1.0,0.0
19197,1.0,0.0,0.0,0.0,1.0,0.0
19198,0.0,1.0,0.0,0.0,1.0,0.0


In [363]:
# after we have generated the dummy columns, we want to drop the original columns from the training data, and replace them with the dummy columns
train_set = fitting_splits.train_set.drop(['sex', 'marriage'], axis=1)
train_set = pd.concat([train_set, df_encoded], axis=1)
train_set

,id,limit_bal,education,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,...,pay_amt6,default,max_delay,freq_delay,sex_1,sex_2,marriage_0,marriage_1,marriage_2,marriage_3
0,13126,400000,1,34,-2,-2,-2,-2,-2,-2,...,0,0,-2,0,1.0,0.0,0.0,1.0,0.0,0.0
1,14636,80000,2,34,0,0,0,0,0,0,...,2000,0,0,6,1.0,0.0,0.0,0.0,1.0,0.0
2,19430,200000,3,49,1,-2,-1,-1,-1,-1,...,0,0,1,1,0.0,1.0,0.0,1.0,0.0,0.0
3,4382,20000,2,41,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,0.0,1.0,0.0,1.0,0.0,0.0
4,7660,70000,1,36,2,0,0,0,0,0,...,0,0,2,6,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,18960,300000,2,30,-1,-1,-1,-1,-1,2,...,7705,0,2,1,0.0,1.0,0.0,0.0,1.0,0.0
19196,254,160000,1,28,0,0,0,0,-1,0,...,10000,0,0,5,0.0,1.0,0.0,0.0,1.0,0.0
19197,11838,30000,2,29,6,5,4,3,2,0,...,0,1,6,6,1.0,0.0,0.0,0.0,1.0,0.0
19198,6120,50000,2,24,2,0,0,2,2,2,...,2200,1,2,6,0.0,1.0,0.0,0.0,1.0,0.0


In [364]:
# now we fit the StandardScaler, in order to standardize all the features
standard_scaler = StandardScaler()
standard_scaler.fit(train_set)
# the above standard scaler, when apply to a dataset with the same columns as the train set, will scale all the numerical features and return a numpy array
standard_scaler.transform(train_set)

StandardScaler()

array([[-0.21449296,  1.77779765, -1.07899959, ...,  1.09375214,
        -1.06692209, -0.10439709],
       [-0.04047304, -0.67618793,  0.18622529, ..., -0.91428392,
         0.93727556, -0.10439709],
       [ 0.51201136,  0.24405666,  1.45145016, ...,  1.09375214,
        -1.06692209, -0.10439709],
       ...,
       [-0.36292848, -1.05962317,  0.18622529, ..., -0.91428392,
         0.93727556, -0.10439709],
       [-1.02189925, -0.90624908,  0.18622529, ..., -0.91428392,
         0.93727556, -0.10439709],
       [-1.18646907, -0.67618793, -1.07899959, ..., -0.91428392,
         0.93727556, -0.10439709]])

Once we have a one hot encoder and a standard scaler fitted on the training data, it is handy to combine all the transformations needed to prepare a dataset with the same schema as the training dataset into a class. This is so that we can apply it to training/validation/test, and any new dataset with the same columns that we might get in the future.

In [365]:
class DataPreparer(object):

    def __init__(self, one_hot_encoder, standard_scaler):
        self.one_hot_encoder = one_hot_encoder
        self.standard_scaler = standard_scaler

    def dummify(self, df):
        vars_to_encode = ['sex', 'marriage']
        df_to_encode = df[vars_to_encode]
        df_encoded = self.one_hot_encoder.transform(df_to_encode).toarray()
        df_encoded = pd.DataFrame(df_encoded)
        df_encoded.columns = self.one_hot_encoder.get_feature_names_out()
        # add the encoded columns and drop the original columns
        df = df.drop(vars_to_encode,axis=1)
        df = pd.concat([df, df_encoded], axis=1)
        return df

    def scale(self, df):
        cols = df.columns
        df = self.standard_scaler.transform(df)
        df = pd.DataFrame(df)
        df.columns = cols
        return df

    def prepare_data(self, df):
        df = df.reset_index(drop=True)
        # first dummify the data
        df = self.dummify(df)
        # then scale it
        df = self.scale(df)
        return df

In [366]:
data_preparer = DataPreparer(one_hot_encoder, standard_scaler)
data_preparer.prepare_data(fitting_splits.train_set).head()
data_preparer.prepare_data(fitting_splits.validation_set).head()

,id,limit_bal,education,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,...,pay_amt6,default,max_delay,freq_delay,sex_1,sex_2,marriage_0,marriage_1,marriage_2,marriage_3
0,-0.214493,1.777798,-1.079000,-0.163603,-1.76117,-1.551696,-1.534925,-1.526049,-1.532662,-1.486580,...,-0.296744,-0.535355,-1.814149,-1.634404,1.239081,-1.239081,-0.039559,1.093752,-1.066922,-0.104397
1,-0.040473,-0.676188,0.186225,-0.163603,0.01807,0.113670,0.142254,0.192989,0.242087,0.259930,...,-0.181969,-0.535355,-0.324056,0.807789,1.239081,-1.239081,-0.039559,-0.914284,0.937276,-0.104397
2,0.512011,0.244057,1.451450,1.464624,0.90769,-1.551696,-0.696335,-0.666530,-0.645288,-0.613325,...,-0.296744,-0.535355,0.420990,-1.227371,-0.807049,0.807049,-0.039559,1.093752,-1.066922,-0.104397
3,-1.222195,-1.136310,0.186225,0.596236,-0.87155,-0.719013,-0.696335,-0.666530,-0.645288,-0.613325,...,-0.296744,-0.535355,-1.069102,-1.634404,-0.807049,0.807049,-0.039559,1.093752,-1.066922,-0.104397
4,-0.844422,-0.752875,-1.079000,0.053494,1.79731,0.113670,0.142254,0.192989,0.242087,0.259930,...,-0.296744,-0.535355,1.166036,0.807789,-0.807049,0.807049,-0.039559,1.093752,-1.066922,-0.104397


,id,limit_bal,education,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,...,pay_amt6,default,max_delay,freq_delay,sex_1,sex_2,marriage_0,marriage_1,marriage_2,marriage_3
0,-0.299083,0.474118,0.186225,0.162042,1.79731,1.779036,1.819434,0.192989,0.242087,0.259930,...,-0.154308,1.867921,1.166036,0.807789,-0.807049,0.807049,-0.039559,1.093752,-1.066922,-0.104397
1,0.531488,-0.906249,-1.079000,0.487688,-1.76117,-1.551696,-1.534925,-1.526049,-1.532662,-1.486580,...,-0.262599,-0.535355,-1.814149,-1.634404,-0.807049,0.807049,-0.039559,1.093752,-1.066922,-0.104397
2,-1.370976,-0.906249,0.186225,-0.272151,0.01807,0.113670,0.142254,0.192989,0.242087,0.259930,...,-0.239357,-0.535355,-0.324056,0.807789,1.239081,-1.239081,-0.039559,1.093752,-1.066922,-0.104397
3,0.417165,-0.906249,-1.079000,-1.249087,0.01807,0.113670,1.819434,0.192989,0.242087,0.259930,...,-0.255425,1.867921,1.166036,0.807789,-0.807049,0.807049,-0.039559,1.093752,-1.066922,-0.104397
4,1.458173,-0.906249,0.186225,-1.357636,-0.87155,0.113670,0.142254,1.912027,2.016836,2.006439,...,-0.296744,-0.535355,1.166036,0.400757,-0.807049,0.807049,-0.039559,-0.914284,0.937276,-0.104397


# Modeling

Now, we can move on to train a support vector classification model.

1. First, we are going to train a default SVC model on the train set, and use it to predict on the test set. This is just to illustrate the code and the steps involved in fitting and predicting with a model using sklearn.
2. Then we are going to evaluate the predictions that the above model generates on the test set. How is the model performing?
3. Then, we are going to:
    - perform a train-validation grid-search in the hyperparameter space of the SVC model (**hyperparameter tuning**)
    - choose the best set of hyperparameters among those explored
    - train a SVC model with the found hyperparameters on train + validation, and predict on test 
4. Finally, we are going to evaluate the predictions of the resulting model at point 3

## Simple train/test model

In [367]:
# get the transformed train set
train_set_transformed = data_preparer.prepare_data(train_test_split.train_set)
X_train = train_set_transformed.drop(['default', 'id'], axis=1) # need to drop the target! otherwise data leakage
y_train = train_test_split.train_set['default'] # take it from the original untransformed dataset
# create the model instance
simple_SVC = SVC(gamma='auto')
# fit the model instance on X_train
simple_SVC.fit(X=X_train, y=y_train)

SVC(gamma='auto')

In [368]:
# now we can use the fitted simple_SVC model to predict on the test dataset!
X_test = data_preparer.prepare_data(train_test_split.test_set).drop(['default', 'id'], axis=1)
y_hat_test = simple_SVC.predict(X_test)
# the result is a vector of predicted classes for the observations in validation, which the model was not trained on!
pd.Series(y_hat_test)

0       0
1       0
2       0
3       0
4       0
       ..
5995    0
5996    1
5997    1
5998    0
5999    0
Length: 6000, dtype: int32

In [369]:
# we can already see that the model is underpredicting the minority class and overpredicting on the majority class
pd.Series(y_hat_test).value_counts()
train_test_split.test_set['default'].value_counts()

0    5345
1     655
dtype: int64

0    4693
1    1307
Name: default, dtype: int64

### Evaluate the model performance

We can now evaluate the classification performance of the model on the test set.
There are various possible metrics that can be useful to evaluate the model performance. In particular, the following can be considered:

- accuracy score: the percentage of examples that was correctly classified (see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
    - pros: simple to interpret and communicate to non-technical people
    - cons: it can be too simple and disguise bad performance as good performance, for instance in the case of imbalanced datasets

- confusion matrix: create the 2 x 2 confusion matrix, and use it to evaluate the model misclassifications:
   - pros: easy to interpret, visual, shows the difference in classification accuracy on the two classes
   - cons: it does not provide a single accuracy metrics, which is useful to compare multiple models against each other

<div>
<img src="img/confusion_matrix.png" width="400">
</div>

- evaluate the precision and recall of the classification separately:
    - precision: the % of the examples classified as positives (defaults) that are actually positive: TP / TP + FP
    - recall: the % of the actually positive examples (defaults) that was classified as positive: TP / TP + FN

- F1-score: the harmonic mean of the precision and recall scores (see above).
   - calculated as TP/(TP + 1/2(FP + FN)). Ranges between 0 and 1.
   - the higher the score, the better the model
   - weights false negatives and false positives equally (which might now always be what you want, as these can have different associated costs)
    - pros: it combines precision and recall into a single score that can be used to evaluate models
    - pros: realiable with imbalanced datasets
    - cons: it can be hard to explain to stakeholders. 

For the present exercise, we are going to use the F1-score to evaluate the classification accuracy. Sklearn has a handy function in the metrics module to calculate the F1 score, the f1_score function.

In [370]:
round(f1_score(train_test_split.test_set['default'], y_hat_test), 2)

0.44

We got 0.45 as an F1 score. Is this good? Is this bad? These are the wrong questions. An f1 score is a **relative** measure of accuracy, i.e., it is useful to compare different models against each other, and determine which one is the better classifier. But it's not useful by itself in isolation.

For instance, consider the simplest possible classification model: the model that classifies everything as 0 (no default). The F1 score of this model then is:

In [371]:
f1_score(train_test_split.test_set['default'], np.zeros(train_test_split.test_set.shape[0]))

0.0

According to the F1-score, this model sucks (it's the worst possible model), even though its accuracy might be quite high because of the imbalanced data. The F1 score is the correct measure here, but it's also telling us that the model above is a better model than this naive model.

Since we have an imbalanced dataset, we might want to do a quick try and see if we can improve the performance by balancing it. The quickest way to do this is to downsample the negative class so as to achieve balance. Let's see what happens.

In [372]:
# check out the distribution of the class in the train set
train_test_split.train_set.groupby('default').size()
# we downsample the negative class, choosing the same number of observations as we have for the positive class
# we then discard data and obtain a balanced dataset
train_set_sampled = train_test_split.train_set.sample(4783, random_state=1234)

# after this the code is the same as above, to fit and predict with the model
train_set_transformed = data_preparer.prepare_data(train_set_sampled)
X_train = train_set_transformed.drop(['default', 'id'], axis=1) # need to drop the target! otherwise data leakage
y_train = train_set_sampled['default'] # take it from the original untransformed dataset
# create the model instance
simple_SVC = SVC(gamma='auto')
# fit the model instance on X_train
simple_SVC.fit(X=X_train, y=y_train)

X_test = data_preparer.prepare_data(train_test_split.test_set).drop(['default', 'id'], axis=1)
y_hat_test = simple_SVC.predict(X_test)
# the result is a vector of predicted classes for the observations in validation, which the model was not trained on!
round(f1_score(train_test_split.test_set['default'], y_hat_test), 2)

default
0    16817
1     4783
dtype: int64

SVC(gamma='auto')

0.41

The f1 score of the above model is only 0.42. In other words, we have lost performance by downsampling, compared to the first basic model. That means that the benefit of making the training set balanced is outweighed by the loss of training examples.
Most likely, an approach like SMOTE (see the EDA notebook) would work better here.

# Hypertuning

In the above, we have fitted and predicted with basically the default SVC model. However, each machine learning model comes with a set of **hyperparameters** that can be tuned. Think of a model as a machine, and of hyperparameters as levers and dials of the machine, which need to be set before it can used. In our case, the hyperparameters must be set before we can train and then predict with the model.

But how should we set these parameters? There is no way to set them from data, in the fitting process. Rather, we need to explore the space of possible values for these hyperparameters, and for each combination that we might want to try, fit a model and evaluate its predictive performance. The combination of hyperparameters values that returns the model with the highest performance is the optimal value of hyperparameters among the combination explored.

Note that model hyperparameters are different from model parameters. The main difference is that the latter are learned during the fitting process, and estimated directly from the data in this process. The model hyperparameters cannot be estimated in this way because they define the "shape" or "class" of the model before the training has taken place. See also https://machinelearningmastery.com/difference-between-a-parameter-and-a-hyperparameter/ for an explanation of the difference.

Often, a model will have many hyperparameters. Not all of them are important, or have the same impact on the performance of the model. So how do you determine which ones to tune? The only way is from experience, or by understanding the mathematics of the model and understanding which parameters matter the most, or by empirical research that has been done by other researchers

For SVMs, the most important parameters are:
- The kernel. This is the actual function that maps the observations to the (higher-dimensional) space. In the above, we used the default radial kernel, which is also that which performs the best most of the times
- for different kernels there might be different hyperparamters that can be tuned. These hyperparamters usually specify the shape of the transformation of the feature space. For the radial kernel, we have two important ones:
- the gamma parameter
- the C ('cost') parameter

You can find a good description of what these parameters control here: https://towardsdatascience.com/svm-hyperparameters-explained-with-visualizations-143e48cb701b, but for now it suffices to understand that these are parameters that control the shape of the model, and their values can have an impact on the model performance.

We then want to tune the gamma and C parameter to see if we get a better SVC model. There are various approach to do this, but the simplest is: **grid search**. 

- This means that we will create a grid with different combinations of values of the two parameters
- For each combination of hyperparameter values we fit a SVC model to the data with those hyperparameter values, and predict on the **validation** dataset
- The model which returns the best f1-score will show us what the best hyperparameters in the grid are
- We can then use these hyperparameters to train a final model on all the fitting data, and predict on the test set

<div>
<img src="img/hyperparameter_tuning.png" width="800">
</div>

In [373]:
C_range = [2 ** -5, 2 ** 15]
gamma_range = [2 ** -15, 2 ** 3]

In [392]:
C_grid = np.linspace(C_range[0], C_range[1], num=10)
gamma_grid = np.linspace(gamma_range[0], gamma_range[1], num=10)

In [393]:
C_grid
gamma_grid

array([3.12500000e-02, 3.64091667e+03, 7.28180208e+03, 1.09226875e+04,
       1.45635729e+04, 1.82044583e+04, 2.18453438e+04, 2.54862292e+04,
       2.91271146e+04, 3.27680000e+04])

array([3.05175781e-05, 8.88916016e-01, 1.77780151e+00, 2.66668701e+00,
       3.55557251e+00, 4.44445801e+00, 5.33334351e+00, 6.22222900e+00,
       7.11111450e+00, 8.00000000e+00])

In [394]:
# will hold the models fitted on train, for each hyperparameter combination
fitted_models = []

X_train = data_preparer.prepare_data(fitting_splits.train_set).drop(['default', 'id'], axis=1)
y_train = fitting_splits.train_set[ 'default']

def fit_svm_model(X_train, y_train, C, gamma):
    # create the model instance with the required parameters
    simple_SVC = SVC(gamma=gamma, C=C)
    # fit the model instance on X_train
    simple_SVC.fit(X=X_train, y=y_train)
    return simple_SVC

# train a model for each hyperparameter combo
print("begin hypertuning")
start_time = datetime.datetime.now()
for C in C_grid:
    for gamma in gamma_grid:
        # fit a model with the given hyperparameter values.
        # A new model will be fitted for each loop cycle
        print(f"fitting model for C: {C} and gamma: {gamma}")
        fitted_model = fit_svm_model(X_train, y_train, C, gamma)
        print(f"fitting complete.")
        # store the fitted model in the fitted_model list
        fitted_models.append(fitted_model)

end_time = datetime.datetime.now()
print(f'hypertuning complete in {round((end_time - start_time).seconds/60, 2)} minutes')

begin hypertuning
fitting model for C: 0.03125 and gamma: 3.0517578125e-05
fitting complete.
fitting model for C: 0.03125 and gamma: 0.888916015625
fitting complete.
fitting model for C: 0.03125 and gamma: 1.777801513671875
fitting complete.
fitting model for C: 0.03125 and gamma: 2.66668701171875
fitting complete.
fitting model for C: 0.03125 and gamma: 3.555572509765625
fitting complete.
fitting model for C: 0.03125 and gamma: 4.4444580078125
fitting complete.
fitting model for C: 0.03125 and gamma: 5.333343505859375
fitting complete.
fitting model for C: 0.03125 and gamma: 6.22222900390625
fitting complete.
fitting model for C: 0.03125 and gamma: 7.111114501953125
fitting complete.
fitting model for C: 0.03125 and gamma: 8.0
fitting complete.
fitting model for C: 3640.9166666666665 and gamma: 3.0517578125e-05
fitting complete.
fitting model for C: 3640.9166666666665 and gamma: 0.888916015625
fitting complete.
fitting model for C: 3640.9166666666665 and gamma: 1.777801513671875
fitti

In [398]:
# store the fitted models as a pickle so they can be analysed later
with open('../dataset/hypertuning_models.pickle', 'wb') as handle:
    pickle.dump(fitted_models, handle)

In [395]:
# evaluate the F1 score of each model on validation, and find the best hyperparameters
X_validation = data_preparer.prepare_data(fitting_splits.validation_set).drop(['default', 'id'], axis=1)
y_validation = fitting_splits.validation_set['default']

def calculate_f1_score(X, y, model):
    y_hat = model.predict(X)
    return f1_score(y, y_hat)

f_scores = [calculate_f1_score(X_validation, y_validation, m) for m in fitted_models]

ValueError: 0.40864440078585457 is not in list

In [399]:
f_scores

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.37237643872714965,
 0.2966666666666667,
 0.24906132665832292,
 0.2219269102990033,
 0.19818562456385203,
 0.18116462976276063,
 0.1652046783625731,
 0.1525804038893044,
 0.13465952563121653,
 0.1254841208365608,
 0.38692461641094067,
 0.2912300055157198,
 0.2485946283572767,
 0.2219269102990033,
 0.19818562456385203,
 0.18116462976276063,
 0.1652046783625731,
 0.1525804038893044,
 0.13465952563121653,
 0.1254841208365608,
 0.39387890884896876,
 0.2866629773104593,
 0.24765478424015008,
 0.2217795484727756,
 0.19818562456385203,
 0.18116462976276063,
 0.1652046783625731,
 0.1525804038893044,
 0.13465952563121653,
 0.1254841208365608,
 0.3952254641909814,
 0.2880886426592798,
 0.24765478424015008,
 0.2217795484727756,
 0.19818562456385203,
 0.18116462976276063,
 0.1652046783625731,
 0.1525804038893044,
 0.13465952563121653,
 0.1254841208365608,
 0.4,
 0.28982300884955753,
 0.2485946283572767,
 0.2217795484727756,
 0.198185624

In [405]:
# choose the model with the best F score value
max_f1_score = max(f_scores)
best_model_index = f_scores.index(max_f1_score)
best_model = fitted_models[best_model_index]

In [409]:
print(f"The best value of C found is {best_model.C}")
print(f"The best value of gamma is {best_model.gamma}")

The best value of C found is 32768.0
The best value of gamma is 3.0517578125e-05


In [415]:
now train a model with the best parameters on train + validation, and predict on test to get the final out-of-sample performance of the model at hand

In [414]:
selected_model = SVC(gamma = best_model.gamma, C = best_model.C)
X_train_validation = pd.concat([X_train, X_validation])
y_train_validation = pd.concat([fitting_splits.train_set['default'], fitting_splits.validation_set['default']])
selected_model.fit(X_train_validation, y_train_validation)

SVC(C=32768.0, gamma=3.0517578125e-05)

In [420]:
# now predict on test with the fitted model to get the final performance measure
X_test = data_preparer.prepare_data(fitting_splits.test_set).drop(['default', 'id'],axis=1)
y_test = fitting_splits.test_set['default']
calculate_f1_score(X_test, y_test, selected_model)

0.40925828511309836

only 0.41 of F1 score. Not great, we didn't manage to beat the default parameters. Some further ideas to improve the SVM model:
- refine the grid around the best parameters found above. Perhaps around the current best parameters found there are better options
- move to random or Bayesian search rather than grid search
- apply the smote technique to generate more examples of the minority class and train the model on the inflated data